<a href="https://colab.research.google.com/github/rmccormick314/GFSAD/blob/main/crop_intensity/GFSAD_Intensity_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Set Up

## a. Requirements

In [1]:
repo_url = "https://raw.githubusercontent.com/GoogleCloudPlatform/python-docs-samples/main/people-and-planet-ai/land-cover-classification"

!wget --quiet {repo_url}/requirements.txt

!pip install --quiet --upgrade pip
!pip install --quiet -r requirements.txt

# Restart the runtime by ending the process.
exit()

## b. Set Project Info

In [1]:
from __future__ import annotations

import os
from google.colab import auth

auth.authenticate_user()

# Please fill in these values.
project = "gfsad-446404"  # @param {type:"string"}
bucket = "lgcip30"  # @param {type:"string"}
location = "us-central1"  # @param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith(
    "gs://"
), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

Updated property [core/project].


## c. Clone GitHub Code

In [2]:
# Now let's get the code from GitHub and navigate to the sample.
!git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git
%cd python-docs-samples/people-and-planet-ai/land-cover-classification

Cloning into 'python-docs-samples'...
remote: Enumerating objects: 118971, done.
remote: Counting objects: 100% (2520/2520), done.
remote: Compressing objects: 100% (575/575), done.
remote: Total 118971 (delta 2229), reused 1945 (delta 1945), pack-reused 116451 (from 2)
Receiving objects: 100% (118971/118971), 243.13 MiB | 24.09 MiB/s, done.
Resolving deltas: 100% (71402/71402), done.
Updating files: 100% (5479/5479), done.
/content/python-docs-samples/people-and-planet-ai/land-cover-classification


## d. Initialize Earth Engine

In [3]:
import ee
import google.auth

credentials, _ = google.auth.default()
ee.Initialize(
    credentials.with_quota_project(None),
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

# II. Create Dataset

## a. Set Parameters

In [7]:
YEAR = "2020" # @param {"type":"string"}
GAEZ = "8" # @param {"type":"string"}

## b. Visualize CDL

In [22]:
import ee
import folium

CLASSIFICATIONS = {
    "Perennial" : "1E90FF",
    "Fodder"    : "00FF00",
    "Dbl. Crop" : "FFD700",
    "Annual"    : "FF4500",
    "Fallow"    : "bfbf77"
}

image = (
    ee.Image("USDA/NASS/CDL/" + YEAR)
    .select("cropland")
    .remap(
        [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 81, 82, 83, 87, 88, 92, 111, 112, 121, 122, 123, 124, 131, 141, 142, 143, 152, 176, 190, 195, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 254],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 5, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 4, 1, 4, 4, 1, 1, 1, 4, 4, 1, 4, 1, 1, 4, 1, 1, 4, 1, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 4, 4, 4, 4, 4, 4, 4, 1, 3]
    )
    .rename("label")
)

vis_params = {
    "max": len(CLASSIFICATIONS),
    "palette": list(CLASSIFICATIONS.values()),
    "bands": ["label"],
}
folium.Map(
    location=(38, -106),
    zoom_start=5,
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)